In [ ]:
import pandas as pd
import numpy as np
import copy
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split as tts
from xgboost import XGBClassifier
train = pd.read_csv("/content/drive/MyDrive/데이콘 고객대출등급 분류/train.csv")
test = pd.read_csv("/content/drive/MyDrive/데이콘 고객대출등급 분류/test.csv")
submission = pd.read_csv('/content/drive/MyDrive/데이콘 고객대출등급 분류/sample_submission.csv')


In [ ]:
train=train.iloc[:,1:-1]
test=test.iloc[:,1:]

In [ ]:
train.value_counts('근로기간')

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
dtype: int64

In [ ]:
train.loc[train['근로기간']=='3','근로기간']='3 years'
test.loc[test['근로기간']=='3','근로기간']='3 years'

In [ ]:
train.loc[train['근로기간']=='1 year','근로기간']='1 years'
test.loc[test['근로기간']=='1 year','근로기간']='1 years'

In [ ]:
train.loc[train['근로기간']=='<1 year','근로기간']='< 1 year'
test.loc[test['근로기간']=='<1 year','근로기간']='< 1 year'

In [ ]:
train.loc[train['근로기간']=='10+years','근로기간']='10+ years'
test.loc[test['근로기간']=='10+years','근로기간']='10+ years'

In [ ]:
train.value_counts('근로기간')

근로기간
10+ years    32481
2 years       8450
< 1 year      8144
3 years       7670
1 years       6305
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
dtype: int64

In [ ]:
train['근로기간'].unique().tolist()

['6 years',
 '10+ years',
 '5 years',
 '8 years',
 'Unknown',
 '9 years',
 '2 years',
 '1 years',
 '3 years',
 '7 years',
 '4 years',
 '< 1 year']

In [ ]:
values_map=dict({'6 years':6,
 '10+ years':10,
 '5 years':5,
 '8 years':8,
 '9 years':9,
 '2 years':2,
 '1 years':1,
 '3 years':3,
 '7 years':7,
 '4 years':4,
 '< 1 year':0,
 'Unknown':np.NaN})

In [ ]:
train['근로기간']=train['근로기간'].map(values_map)
test['근로기간']=test['근로기간'].map(values_map)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown='ignore')
col=['대출기간','주택소유상태','대출목적','연체계좌수','최근_2년간_연체_횟수','총계좌수']

In [ ]:
enc.fit(train[col])

OneHotEncoder(handle_unknown='ignore')

In [ ]:
encoded_data = enc.fit_transform(train[col]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names_out(col))
train= train.drop(columns=['주택소유상태','대출목적']).join(encoded_df)
train['대출기간'] = train['대출기간'].str.strip().replace('months', '', regex=True).astype(float)

train

,대출금액,대출기간,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,총계좌수_106,총계좌수_109,총계좌수_110,총계좌수_112,총계좌수_125,총계좌수_127,총계좌수_130,총계좌수_140,총계좌수_151,총계좌수_169
0,12480000,36.0,6.0,72000000,18.90,15,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14400000,60.0,10.0,130800000,22.33,21,0,373572,234060.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12000000,36.0,5.0,96000000,8.60,14,0,928644,151944.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14400000,36.0,8.0,132000000,15.09,15,0,325824,153108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18000000,60.0,NaN,71736000,25.39,19,0,228540,148956.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36.0,10.0,210000000,9.33,33,0,974580,492168.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96290,28800000,60.0,10.0,132000000,5.16,25,0,583728,855084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96291,14400000,36.0,1.0,84000000,11.24,22,0,1489128,241236.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96292,15600000,36.0,5.0,66330000,17.30,21,2,1378368,818076.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
encoded_test_data = enc.transform(test[col]).toarray()
encoded_test_df = pd.DataFrame(encoded_test_data, columns=enc.get_feature_names_out(col))
test= test.drop(columns=['주택소유상태','대출목적']).join(encoded_test_df)
test['대출기간'] = test['대출기간'].str.strip().replace('months', '', regex=True).astype(float)

test


,대출금액,대출기간,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,총계좌수_106,총계좌수_109,총계좌수_110,총계좌수_112,총계좌수_125,총계좌수_127,총계좌수_130,총계좌수_140,총계좌수_151,총계좌수_169
0,16800000,36.0,8.0,132000000,19.64,12,0,394692,146604.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8400000,36.0,5.0,89971200,15.84,25,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17280000,36.0,6.0,150000000,8.41,20,0,1786980,281820.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14400000,36.0,5.0,66000000,13.72,30,1,669024,281724.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,27600000,36.0,5.0,55200000,30.50,12,0,1250052,614844.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,30000000,36.0,3.0,78000000,22.08,27,2,1307532,763380.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64193,30000000,60.0,10.0,109200000,12.06,26,0,960612,1245252.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64194,6120000,36.0,10.0,39600000,28.80,33,0,131520,80880.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64195,11520000,36.0,10.0,66000000,25.44,41,1,1339536,601872.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.isnull().sum()[train.isnull().sum()>0],test.isnull().sum()[test.isnull().sum()>0]

(근로기간    5671
 dtype: int64,
 근로기간    3862
 dtype: int64)

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier

In [ ]:
train_data_with_values = train.dropna(subset=['근로기간'])
train_data_missing_values = train[train['근로기간'].isna()]

In [ ]:
X = train_data_with_values.drop(['근로기간'], axis=1)
y = train_data_with_values['근로기간']

In [ ]:
C = train_data_with_values.drop(['근로기간'], axis=1)
d = train_data_with_values['근로기간']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
C_train, C_valid, d_train, d_valid = train_test_split(C, d, test_size=0.2, random_state=42)

In [ ]:
cat_clf = CatBoostClassifier(silent=True)
lgb_clf = LGBMClassifier()
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
class CustomCatBoostClassifier(CatBoostClassifier):
    def predict(self, X, **kwargs):
        predictions = super().predict(X, **kwargs)

        return np.ravel(predictions)

In [ ]:
cat_clf = CustomCatBoostClassifier(silent=True)

lgb_clf = LGBMClassifier()
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('cat', cat_clf), ('lgb', lgb_clf), ('xgb', xgb_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1627
[LightGBM] [Info] Number of data points in the train set: 72498, number of used features: 111
[LightGBM] [Info] Start training from score -2.408071
[LightGBM] [Info] Start training from score -2.678129
[LightGBM] [Info] Start training from score -2.370615
[LightGBM] [Info] Start training from score -2.474123
[LightGBM] [Info] Start training from score -2.787290
[LightGBM] [Info] Start training from score -2.773058
[LightGBM] [Info] Start training from score -3.157007
[LightGBM] [Info] Start training from score -3.172360
[LightGBM] [Info] Start training from score -2.916193
[LightGBM] [Info] Start training from score -3.191971
[LightGBM] [Info] Start tr

VotingClassifier(estimators=[('cat',
                              <__main__.CustomCatBoostClassifier object at 0x7b9a207d1990>),
                             ('lgb', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_type...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))])

In [ ]:
train_data_missing_values_prepared = train_data_missing_values.drop(['근로기간'], axis=1)
train_missing_values_predictions= voting_clf.predict(train_data_missing_values_prepared)

In [ ]:
voting_clf.predict(X_valid)

array([10., 10., 10., ...,  2., 10., 10.])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print(classification_report(y_valid,voting_clf.predict(X_valid)))
print(accuracy_score(y_valid,voting_clf.predict(X_valid)))

              precision    recall  f1-score   support

         0.0       0.17      0.07      0.09      1620
         1.0       0.10      0.01      0.01      1325
         2.0       0.15      0.02      0.03      1677
         3.0       0.10      0.01      0.01      1563
         4.0       0.12      0.00      0.00      1123
         5.0       0.17      0.00      0.00      1136
         6.0       1.00      0.00      0.00       789
         7.0       0.00      0.00      0.00       776
         8.0       0.00      0.00      0.00       963
         9.0       0.00      0.00      0.00       765
        10.0       0.36      0.97      0.53      6388

    accuracy                           0.35     18125
   macro avg       0.20      0.10      0.06     18125
weighted avg       0.23      0.35      0.20     18125

0.3519448275862069


In [ ]:
train.loc[train['근로기간'].isna(), '근로기간'] = train_missing_values_predictions
train

,대출금액,대출기간,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,총계좌수_106,총계좌수_109,총계좌수_110,총계좌수_112,총계좌수_125,총계좌수_127,총계좌수_130,총계좌수_140,총계좌수_151,총계좌수_169
0,12480000,36.0,6.0,72000000,18.90,15,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14400000,60.0,10.0,130800000,22.33,21,0,373572,234060.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12000000,36.0,5.0,96000000,8.60,14,0,928644,151944.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14400000,36.0,8.0,132000000,15.09,15,0,325824,153108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,18000000,60.0,10.0,71736000,25.39,19,0,228540,148956.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36.0,10.0,210000000,9.33,33,0,974580,492168.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96290,28800000,60.0,10.0,132000000,5.16,25,0,583728,855084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96291,14400000,36.0,1.0,84000000,11.24,22,0,1489128,241236.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96292,15600000,36.0,5.0,66330000,17.30,21,2,1378368,818076.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.isnull().sum().sum()

0

In [ ]:
test.isnull().sum().sum()

3862

In [ ]:
test_data_with_values = test.dropna(subset=['근로기간'])
test_data_missing_values = test[test['근로기간'].isna()]

In [ ]:
voting_clf.fit(C_train, d_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1627
[LightGBM] [Info] Number of data points in the train set: 72498, number of used features: 111
[LightGBM] [Info] Start training from score -2.408071
[LightGBM] [Info] Start training from score -2.678129
[LightGBM] [Info] Start training from score -2.370615
[LightGBM] [Info] Start training from score -2.474123
[LightGBM] [Info] Start training from score -2.787290
[LightGBM] [Info] Start training from score -2.773058
[LightGBM] [Info] Start training from score -3.157007
[LightGBM] [Info] Start training from score -3.172360
[LightGBM] [Info] Start training from score -2.916193
[LightGBM] [Info] Start training from score -3.191971
[LightGBM] [Info] Start tr

VotingClassifier(estimators=[('cat',
                              <__main__.CustomCatBoostClassifier object at 0x7b9a207d1990>),
                             ('lgb', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_type...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))])

In [ ]:
test_data_missing_values_prepared = test_data_missing_values.drop(['근로기간'], axis=1)
test_missing_values_predictions= voting_clf.predict(test_data_missing_values_prepared)

In [ ]:
test_missing_values_predictions= voting_clf.predict(test_data_missing_values_prepared)

In [ ]:
voting_clf.predict(C_valid)

array([10., 10., 10., ...,  2., 10., 10.])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print(classification_report(d_valid,voting_clf.predict(C_valid)))
print(accuracy_score(d_valid,voting_clf.predict(C_valid)))

              precision    recall  f1-score   support

         0.0       0.17      0.07      0.09      1620
         1.0       0.10      0.01      0.01      1325
         2.0       0.15      0.02      0.03      1677
         3.0       0.10      0.01      0.01      1563
         4.0       0.12      0.00      0.00      1123
         5.0       0.17      0.00      0.00      1136
         6.0       1.00      0.00      0.00       789
         7.0       0.00      0.00      0.00       776
         8.0       0.00      0.00      0.00       963
         9.0       0.00      0.00      0.00       765
        10.0       0.36      0.97      0.53      6388

    accuracy                           0.35     18125
   macro avg       0.20      0.10      0.06     18125
weighted avg       0.23      0.35      0.20     18125

0.3519448275862069


In [ ]:
test.loc[test['근로기간'].isna(), '근로기간'] = test_missing_values_predictions
test

,대출금액,대출기간,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,총계좌수_106,총계좌수_109,총계좌수_110,총계좌수_112,총계좌수_125,총계좌수_127,총계좌수_130,총계좌수_140,총계좌수_151,총계좌수_169
0,16800000,36.0,8.0,132000000,19.64,12,0,394692,146604.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8400000,36.0,5.0,89971200,15.84,25,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17280000,36.0,6.0,150000000,8.41,20,0,1786980,281820.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14400000,36.0,5.0,66000000,13.72,30,1,669024,281724.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,27600000,36.0,5.0,55200000,30.50,12,0,1250052,614844.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,30000000,36.0,3.0,78000000,22.08,27,2,1307532,763380.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64193,30000000,60.0,10.0,109200000,12.06,26,0,960612,1245252.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64194,6120000,36.0,10.0,39600000,28.80,33,0,131520,80880.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64195,11520000,36.0,10.0,66000000,25.44,41,1,1339536,601872.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test.isnull().sum().sum()

0

In [ ]:
train1=pd.read_csv("/content/drive/MyDrive/데이콘 고객대출등급 분류/train.csv")
train1

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [ ]:
train['대출등급']=train1['대출등급']

In [ ]:
test1=pd.read_csv(("/content/drive/MyDrive/데이콘 고객대출등급 분류/test.csv"))

In [ ]:
train

,대출금액,대출기간,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,총계좌수_109,총계좌수_110,총계좌수_112,총계좌수_125,총계좌수_127,총계좌수_130,총계좌수_140,총계좌수_151,총계좌수_169,대출등급
0,12480000,36.0,6.0,72000000,18.90,15,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C
1,14400000,60.0,10.0,130800000,22.33,21,0,373572,234060.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B
2,12000000,36.0,5.0,96000000,8.60,14,0,928644,151944.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A
3,14400000,36.0,8.0,132000000,15.09,15,0,325824,153108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C
4,18000000,60.0,10.0,71736000,25.39,19,0,228540,148956.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36.0,10.0,210000000,9.33,33,0,974580,492168.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C
96290,28800000,60.0,10.0,132000000,5.16,25,0,583728,855084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,E
96291,14400000,36.0,1.0,84000000,11.24,22,0,1489128,241236.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A
96292,15600000,36.0,5.0,66330000,17.30,21,2,1378368,818076.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler

target_variable = '대출등급'

# 불균형한 클래스에 대응하기 위한 RandomOverSampler를 초기화합니다.
oversampler = RandomOverSampler(random_state=42)

# 타겟 변수를 레이블 인코딩합니다.
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train[target_variable])

# 데이터를 특성 (X) 및 타겟 변수 (y)로 분할합니다.
X = train.drop(target_variable, axis=1)
y = y_encoded

# RandomOverSampler를 사용하여 오버샘플링합니다.
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# 5-폴드 교차 검증을 위한 객체를 초기화합니다.
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증을 통한 모델 평가
for train_idx, val_idx in kf.split(X_resampled, y_resampled):
    X_train, X_val = X_resampled.iloc[train_idx], X_resampled.iloc[val_idx]
    y_train, y_val = y_resampled[train_idx], y_resampled[val_idx]

    # Imbalance-XGBoost를 사용하여 XGBClassifier를 초기화합니다.
    xgb_model = XGBClassifier(
        n_estimators=1400,
        max_depth=8,
        learning_rate=0.09,
        objective='multi:softmax',  # 다중 클래스 분류를 위한 목적 함수
        num_class=len(np.unique(y_resampled))  # 클래스의 개수
    )

    # 모델을 훈련합니다.
    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

    # 검증 세트에 대한 예측을 수행합니다.
    predictions_encoded = xgb_model.predict(X_val)

    # 마이크로 F1 점수를 계산합니다.
    micro_f1 = f1_score(y_val, predictions_encoded, average='micro')

    # 마이크로 F1 점수를 출력하거나 필요한 대로 사용합니다.
    print("마이크로 F1 점수:", micro_f1)

# 최종 모델을 훈련합니다.
xgb_model.fit(X_resampled, y_resampled, eval_set=[(X_val, y_val)], verbose=100)

# 테스트 세트에 대한 예측을 수행합니다.
test_predictions_encoded = xgb_model.predict(test)

# 예측된 숫자 레이블을 원래 클래스 레이블로 디코딩합니다.
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

[0]	validation_0-mlogloss:1.81282
[100]	validation_0-mlogloss:0.37558
[200]	validation_0-mlogloss:0.27831
[300]	validation_0-mlogloss:0.23334
[400]	validation_0-mlogloss:0.20877
[500]	validation_0-mlogloss:0.19154
[600]	validation_0-mlogloss:0.17814
[700]	validation_0-mlogloss:0.16889
[800]	validation_0-mlogloss:0.16119
[900]	validation_0-mlogloss:0.15599
[1000]	validation_0-mlogloss:0.15177
[1100]	validation_0-mlogloss:0.14842
[1200]	validation_0-mlogloss:0.14598
[1300]	validation_0-mlogloss:0.14422
[1399]	validation_0-mlogloss:0.14301
마이크로 F1 점수: 0.9523349196906603
[0]	validation_0-mlogloss:1.81280
[100]	validation_0-mlogloss:0.36433
[200]	validation_0-mlogloss:0.27376
[300]	validation_0-mlogloss:0.23098
[400]	validation_0-mlogloss:0.20429
[500]	validation_0-mlogloss:0.18633
[600]	validation_0-mlogloss:0.17426
[700]	validation_0-mlogloss:0.16445
[800]	validation_0-mlogloss:0.15698
[900]	validation_0-mlogloss:0.15125
[1000]	validation_0-mlogloss:0.14706
[1100]	validation_0-mlogloss:0.

In [ ]:
submission['대출등급'] = test_predictions

In [ ]:
submission

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [ ]:
submission.to_csv('/content/drive/MyDrive/데이콘 고객대출등급 분류/sub_with_xgb_fillna_1.csv', index = False)

In [ ]:
'''def ordinal_encoding(train_df, test_df, categorical_columns):
    from sklearn.preprocessing import OrdinalEncoder
    train, test = train_df.copy(), test_df.copy()
    data = {}
    for col in categorical_columns:
        ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                                         unknown_value=-1)
        ordinal_encoder.fit(train[col].values.reshape(-1, 1))
        train[col] = ordinal_encoder.transform(train[col].values.reshape(-1, 1)).reshape(-1)
        if col in test:
            test[col] = ordinal_encoder.transform(test[col].values.reshape(-1, 1)).reshape(-1)
        data[col] = ordinal_encoder
    return train, test, data

def sep_ml_xy(df, target):
    y = df[target]
    x = df.drop(columns=target)
    return x, y

def ml_train_valid(model, metric, metric_options, train_data, train_target, test_data, test_target):
    model = model.fit(train_data, train_target)
    pred = model.predict(test_data)
    evaluate = metric(test_target, pred, **metric_options)
    return pred, evaluate, model

def ml_predict(model, test_data):
    pred = model.predict(test_data)
    return pred

train = train.drop(columns=['ID'])
test = test.drop(columns=['ID'])
for span in ["대출기간", "근로기간"]:
    train[span] = train[span].apply(lambda x: int(only_int(x)))
    test[span] = test[span].apply(lambda x: int(only_int(x)))

categorical_columns = extract_categorical_columns(train)
train, test, ord_dict = ordinal_encoding(train, test, categorical_columns)
train_x, train_y = sep_ml_xy(train, "대출등급")
train_x, valid_x, train_y, valid_y = tts(train_x, train_y, train_size=0.8, shuffle=True, random_state=0)
model = XGBClassifier()
_, evaluate, model = ml_train_valid(model, f1_score, {"average": "macro"},
                                      train_x, train_y, valid_x, valid_y)
print("valid score:", evaluate)
pred = ml_predict(model, test)
submission['대출등급'] = ord_dict["대출등급"].inverse_transform(pred.reshape(-1, 1)).reshape(-1)
print(submission)
submission.to_csv('/content/drive/MyDrive/데이콘 고객대출등급 분류/XGB.csv', index=False)
'''